In [9]:
import pandas as pd

In [10]:
diamonds_train = pd.read_csv('data/diamonds_train.csv')
diamonds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40455 non-null  float64
 1   cut      40455 non-null  object 
 2   color    40455 non-null  object 
 3   clarity  40455 non-null  object 
 4   depth    40455 non-null  float64
 5   table    40455 non-null  float64
 6   price    40455 non-null  int64  
 7   x        40455 non-null  float64
 8   y        40455 non-null  float64
 9   z        40455 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.1+ MB


In [11]:
diamonds_predict = pd.read_csv('data/diamonds_test.csv')
diamonds_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       13485 non-null  int64  
 1   carat    13485 non-null  float64
 2   cut      13485 non-null  object 
 3   color    13485 non-null  object 
 4   clarity  13485 non-null  object 
 5   depth    13485 non-null  float64
 6   table    13485 non-null  float64
 7   x        13485 non-null  float64
 8   y        13485 non-null  float64
 9   z        13485 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 1.0+ MB


## GETTING RID OF 0 IN xyz

### TRAIN

In [12]:
diamonds_train = diamonds_train[(diamonds_train['x'] != 0) & (diamonds_train['y'] != 0) & (diamonds_train['z'] != 0)]
diamonds_train.shape

(40439, 10)

### TEST

Celda no ejecutada porque al eliminar rows en el csv de test, da fallo la submission

diamonds_predict = diamonds_predict[(diamonds_predict['x'] != 0) & (diamonds_predict['y'] != 0) & (diamonds_predict['z'] != 0)]
diamonds_predict.info()

## GET RID OF OUTLIERS

Tried with 75-25, not a good idea shape=(6465, 10), we gonna try with some less outliers

In [13]:
# Hay un diamante con un depth de 79
diamonds_train = diamonds_train[(diamonds_train["depth"]<75) & (diamonds_train["depth"]>45)]

# Hay 3 mayores de 75 y dos menores de 45
diamonds_train = diamonds_train[(diamonds_train["table"]<75) & (diamonds_train["table"]>45)]

diamonds_train = diamonds_train[(diamonds_train["x"]<10)] 
#solo dropeamos las que sean x<10 porque coincide con 'y' en sus dos únicos valores por encima de 10 y no dropeamos nada de z porque hay bastantes valores en torno a 6

In [14]:
diamonds_train.shape

(40425, 10)

In [15]:
cat_features = ['cut', 'color', 'clarity']
num_features = ['carat', 'table', 'depth', 'x', 'y', 'z']
predict_num_feat = ['id','carat', 'table', 'depth', 'x', 'y', 'z']
target = 'price'

In [16]:
for cat in cat_features:
    diamonds_train[cat_features] = diamonds_train[cat_features].astype('category')
    diamonds_predict[cat_features] = diamonds_predict[cat_features].astype('category')

In [17]:
diamonds_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40425 entries, 0 to 40454
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    40425 non-null  float64 
 1   cut      40425 non-null  category
 2   color    40425 non-null  category
 3   clarity  40425 non-null  category
 4   depth    40425 non-null  float64 
 5   table    40425 non-null  float64 
 6   price    40425 non-null  int64   
 7   x        40425 non-null  float64 
 8   y        40425 non-null  float64 
 9   z        40425 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 2.6 MB


## Get final df

In [18]:
cat_df_train = pd.get_dummies(diamonds_train[cat_features])
num_df_train = diamonds_train[num_features]


In [19]:
train_df = pd.concat([cat_df_train, num_df_train, diamonds_train[target]], axis=1)

In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40425 entries, 0 to 40454
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cut_Fair       40425 non-null  uint8  
 1   cut_Good       40425 non-null  uint8  
 2   cut_Ideal      40425 non-null  uint8  
 3   cut_Premium    40425 non-null  uint8  
 4   cut_Very Good  40425 non-null  uint8  
 5   color_D        40425 non-null  uint8  
 6   color_E        40425 non-null  uint8  
 7   color_F        40425 non-null  uint8  
 8   color_G        40425 non-null  uint8  
 9   color_H        40425 non-null  uint8  
 10  color_I        40425 non-null  uint8  
 11  color_J        40425 non-null  uint8  
 12  clarity_I1     40425 non-null  uint8  
 13  clarity_IF     40425 non-null  uint8  
 14  clarity_SI1    40425 non-null  uint8  
 15  clarity_SI2    40425 non-null  uint8  
 16  clarity_VS1    40425 non-null  uint8  
 17  clarity_VS2    40425 non-null  uint8  
 18  clarit


### Predict dataframe

In [21]:
cat_df_predict = pd.get_dummies(diamonds_predict[cat_features])
num_df_predict = diamonds_predict[predict_num_feat]

In [22]:
predict_df = pd.concat([cat_df_predict, num_df_predict], axis=1)
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cut_Fair       13485 non-null  uint8  
 1   cut_Good       13485 non-null  uint8  
 2   cut_Ideal      13485 non-null  uint8  
 3   cut_Premium    13485 non-null  uint8  
 4   cut_Very Good  13485 non-null  uint8  
 5   color_D        13485 non-null  uint8  
 6   color_E        13485 non-null  uint8  
 7   color_F        13485 non-null  uint8  
 8   color_G        13485 non-null  uint8  
 9   color_H        13485 non-null  uint8  
 10  color_I        13485 non-null  uint8  
 11  color_J        13485 non-null  uint8  
 12  clarity_I1     13485 non-null  uint8  
 13  clarity_IF     13485 non-null  uint8  
 14  clarity_SI1    13485 non-null  uint8  
 15  clarity_SI2    13485 non-null  uint8  
 16  clarity_VS1    13485 non-null  uint8  
 17  clarity_VS2    13485 non-null  uint8  
 18  clarit

## Splitting our train dataframe to fit and predict

In [154]:
feat = cat_df_train.columns.tolist() + num_df_train.columns.tolist()
train_df[feat] ## df sin precio

,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,...,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,carat,table,depth,x,y,z
0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1.21,58.0,62.4,6.83,6.79,4.25
1,0,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0.32,57.0,63.0,4.35,4.38,2.75
2,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0.71,55.0,65.5,5.62,5.53,3.65
3,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0.41,56.0,63.8,4.68,4.72,3.00
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1.02,59.0,60.5,6.55,6.51,3.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,1.34,57.0,62.7,7.10,7.04,4.43
40451,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,2.02,60.0,57.1,8.31,8.25,4.73
40452,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1.01,56.0,62.7,6.37,6.42,4.01
40453,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0.33,54.3,61.9,4.45,4.47,2.76


In [155]:
train_df[target] ## series with price

0         4268
1          505
2         2686
3          738
4         4882
         ...  
40450    10070
40451    12615
40452     5457
40453      456
40454     6232
Name: price, Length: 40439, dtype: int64

## ESCALAR

In [156]:
from sklearn.preprocessing import StandardScaler

In [157]:
scaler = StandardScaler()

In [158]:
X = scaler.fit_transform(train_df[feat])
y= train_df[target]

## MODEL

In [159]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

### TRAIN TEST SPLIT

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### DECISION TREE

In [160]:
model = DecisionTreeRegressor()

In [165]:
model.fit(X_train, y_train)
crossval_score = cross_val_score(model, X_train, y_train, cv=10)
crossval_score

array([0.96496677, 0.9675275 , 0.96286941, 0.96284244, 0.96719999,
       0.9636627 , 0.960724  , 0.96094883, 0.96327762, 0.96173331])

In [161]:
model.fit(X, y)

DecisionTreeRegressor()

## ESCALAR PREDICT

In [166]:
X_predict = scaler.transform(predict_df[feat])
predictions = model.predict(X_predict)
diamonds_id = predict_df['id']

In [167]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cut_Fair       13485 non-null  uint8  
 1   cut_Good       13485 non-null  uint8  
 2   cut_Ideal      13485 non-null  uint8  
 3   cut_Premium    13485 non-null  uint8  
 4   cut_Very Good  13485 non-null  uint8  
 5   color_D        13485 non-null  uint8  
 6   color_E        13485 non-null  uint8  
 7   color_F        13485 non-null  uint8  
 8   color_G        13485 non-null  uint8  
 9   color_H        13485 non-null  uint8  
 10  color_I        13485 non-null  uint8  
 11  color_J        13485 non-null  uint8  
 12  clarity_I1     13485 non-null  uint8  
 13  clarity_IF     13485 non-null  uint8  
 14  clarity_SI1    13485 non-null  uint8  
 15  clarity_SI2    13485 non-null  uint8  
 16  clarity_VS1    13485 non-null  uint8  
 17  clarity_VS2    13485 non-null  uint8  
 18  clarit

### GENERAR DF

In [168]:
submission = pd.DataFrame({'id': diamonds_id, 'price': predictions})

In [169]:
submission.shape

(13485, 2)

In [170]:
submission.to_csv('./data/submission.csv', index=False)